In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from functools import reduce
from datetime import datetime
import tweepy 
import json 
import re
import seaborn as sns

In [20]:
pd.set_option('display.max_colwidth', -1)

In [21]:
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")

twitter_archive.set_index("tweet_id", inplace = True)


FileNotFoundError: File b'twitter-archive-enhanced.csv' does not exist

In [ ]:
twitter_archive.head()

In [ ]:
twitter_archive.info()

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv' 
 predictions = requests.get(url)

In [ ]:
file_name = url.split('/')[-1]

with open(filename, 'wb') as file:
     file.write(response.content)

In [ ]:
df_image = pd.read_csv('image-predictions.tsv',sep='\t')

In [ ]:
image_prediction.set_index("tweet_id", inplace = True)
image_prediction.head()

In [ ]:
df_list = []

with open('tweet_json.txt', 'r') as file:
    for line in file:
        print(type(line))
        tweet = json.loads(line)
        print(type(tweet))
        break

In [ ]:
df_list = []

with open('tweet_json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        fav_count = tweet['favorite_count']
        df_list.append({'tweet_id':tweet_id,
                       'retweet_count': retweet_count,
                       'favorite_count': fav_count})
        
api_df_now = pd.DataFrame(df_list)
api_df_now.head()

In [ ]:
df_list = []

with open('tweet_json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        fav_count = tweet['favorite_count']
        user_count = tweet['user']['followers_count']
        df_list.append({'tweet_id':tweet_id,
                       'retweet_count': retweet_count,
                       'favorite_count': fav_count,
                       'user_count': user_count})
        
api_df_now = pd.DataFrame(df_list)
api_df_now.sample(10)